In [68]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [69]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=300)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\shiva_coding\\Era 2.0\\agents\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=300))

In [70]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=300)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=300))

In [71]:
tools=[arxiv,wiki]

In [72]:
#costom tools{Rag rools}

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [73]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings


loader=WebBaseLoader("https://www.ibm.com/think/topics/ai-agents")
documents=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200  ).split_documents(documents)
vectordb=FAISS.from_documents(documents,embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))

retriever=vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021F26ACE630>, search_kwargs={})

In [80]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,name="agent_IBM",description="search the web for agents information")

In [81]:
tools=[retriever_tool,arxiv,wiki]
tools

[Tool(name='agent_IBM', description='search the web for agents information', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000021F2B4FE840>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021F26ACE630>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000021F2B4FF9C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021F26ACE630>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_

In [ ]:
from langchain import hub
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
import os

llm=ChatGroq(model="llama3-8b-8192",api_key=os.getenv("GROQ_API_KEY"))
# prompt=hub.pull("hwchase17/openai-functions-agent")
# prompt.messages

prompt = ChatPromptTemplate.from_messages([
    ("system","""You are a helpful assistant that can answer questions based on the chat history.
        If the question is not related to the chat history, ask for more clarification.
        Try to give every information that relates the question in a structured way. 
        If you don't know the answer, say "I don't know."
    """),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("user","{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])




In [88]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [89]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [96]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# Message history store
message_history_store = {}

def get_session_history(session_id):
    if session_id not in message_history_store:
        message_history_store[session_id] = ChatMessageHistory()
    return message_history_store[session_id]

# Wrap agent executor
runnable_agent = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

# Use agent (automated history)
session_id = "user-123"

response = runnable_agent.invoke({"input": "What is Agent?"}, config={"configurable": {"session_id": session_id}})
print(response["output"])





> Entering new AgentExecutor chain...

Invoking: `agent_IBM` with `{'query': 'What is Agent?'}`


What Are AI Agents? | IBM































































































                                   



  
    What are AI agents?






    


                               






























AI Agents



Welcome





Caret right

Introduction




Overview




AI agents vs AI assistants




Agentic AI




Agentic AI vs generative AI




Types of AI agents







Caret right

Components




Overview




Perception




Reasoning




Memory




Planning





Caret right

Tool calling




What is tool calling?




Tutorial: Ollama tool calling






Communication




Learning




Agentic workflows







Caret right

Architecture




Overview




ReAct





Caret right

ReWOO




Tutorial: ReWOO reasoning agent







Caret right

AI agent orchestration




What is agent orchestration?




Tutorial: Agent orchestration





In [97]:
response = runnable_agent.invoke({"input": "How does it works?"}, config={"configurable": {"session_id": session_id}})
print(response["output"])



> Entering new AgentExecutor chain...
I can provide a direct answer:

An AI agent works by using natural language processing (NLP) and machine learning algorithms to understand user requests, identify relevant information, and generate responses. The agent can also use external tools and services, such as search engines, databases, or APIs, to retrieve additional information and provide more accurate responses.

> Finished chain.
I can provide a direct answer:

An AI agent works by using natural language processing (NLP) and machine learning algorithms to understand user requests, identify relevant information, and generate responses. The agent can also use external tools and services, such as search engines, databases, or APIs, to retrieve additional information and provide more accurate responses.


In [98]:
response = runnable_agent.invoke({"input": "How did they impact the world of ai?"}, config={"configurable": {"session_id": session_id}})
print(response["output"])



> Entering new AgentExecutor chain...
I can provide a direct answer:

Artificial intelligence (AI) agents have significantly impacted the world of AI by enabling the development of more sophisticated and autonomous systems. They have improved the ability of AI systems to learn, reason, and interact with humans, leading to breakthroughs in areas such as natural language processing, computer vision, and expert systems. AI agents have also enabled the creation of more advanced AI applications, such as personal assistants, chatbots, and robots, which are transforming industries and revolutionizing the way we live and work.

> Finished chain.
I can provide a direct answer:

Artificial intelligence (AI) agents have significantly impacted the world of AI by enabling the development of more sophisticated and autonomous systems. They have improved the ability of AI systems to learn, reason, and interact with humans, leading to breakthroughs in areas such as natural language processing, comput